In [ ]:
import torch, torch.nn as nn, torch.nn.functional as F
# from midi_utils import MidiFile, load_midi_files
# from importlib import reload
# reload(midi_utils) # To reflect the changes.

In [ ]:
import midi_utils

from importlib import reload
reload(midi_utils) # To reflect the changes.

MidiFile = midi_utils.MidiFile
load_midi_files = midi_utils.load_midi_files

In [ ]:
midi_file = MidiFile.from_filepath("./datasets/jazz_kaggle/2ndMovementOfSinisterFootwear.mid", sequence_len=4)
assert len(midi_file.messages()) != 0

In [ ]:
notes = midi_file.notes()

t  = midi_file.tensor()

midi_iter = iter(midi_file)

for input, target in midi_iter:
    print(input.shape)
    break

In [ ]:
midi_files = [midi_file]


FEATURE_SIZE = 21
HIDDEN_SIZE = 16

# x.shape => (1, FEATURE_SIZE)
eps = 0.01
Wxh = torch.randn((FEATURE_SIZE, HIDDEN_SIZE)) * eps # x @ Wx => (1, HIDDEN_SIZE)
Whh = torch.randn((HIDDEN_SIZE, HIDDEN_SIZE)) * eps
bh = torch.randn((1, HIDDEN_SIZE)) * eps

Why = torch.randn(HIDDEN_SIZE, FEATURE_SIZE) * eps
by = torch.randn(1, FEATURE_SIZE) * eps

hs_init = torch.rand((1, HIDDEN_SIZE)) * eps

def loss_fn(input, target, hs_prev):
    hs, ys = {}, {}
    hs[-1] = torch.clone(hs_prev)

    for i, x in enumerate(input):
        hs[i] =  torch.tanh((x @ Wxh) + (hs[i-1] @ Whh + bh))
        ys[i] = hs[i] @ Why + by
        # loss = ???

    return hs[len(input)-1], ys

for midi_file in midi_files:
    for input, target in midi_file:
        hs, _ = loss_fn(input, target, hs_init)
        print(hs)